In [21]:
import pandas as pd
from datetime import datetime
from pandas import DataFrame
from tabulate import tabulate
from IPython.core.display import HTML
from decimal import Decimal
import time
import json
import os

In [22]:
template_head = '{{% echarts 600 \'95%\' %}}\noption = {{\n    backgroundColor: \'#2c343c\', title: [{{text: \'{0}\', left: \'center\', top: 20, textStyle: {{color: \'#ccc\'}}}}, {1}, {2}, {3}, {4}], tooltip: {{trigger: \'item\'}}, visualMap: {{show: false, min: 1000, max: 5000, inRange: {{colorLightness: [0.8, 0.2]}}}},\n    series: [\n\n    ]\n}};\n{{% endecharts %}}'
template_sub_text = '{{subtext: \'{0}\', left: \'{1}\', top: \'{2}\', textAlign: \'center\', fontSize: 20, textStyle: {{color: \'#ccc\'}}}}'
template_series = '{{name: \'详细信息\', type: \'pie\', radius: \'20%\', center: {0}, {1}, roseType: \'radius\', label: {{color: \'rgba(255, 255, 255, 0.3)\'}}, labelLine: {{lineStyle: {{color: \'rgba(255, 255, 255, 0.3)\'}}, smooth: 0.2, length: 10, length2: 20}}, itemStyle: {{color: \'{2}\', shadowBlur: 200, shadowColor: \'rgba(0, 0, 0, 0.5)\'}}, animationType: \'scale\', animationEasing: \'elasticOut\', animationDelay: function (idx) {{return Math.random() * 200;}}}},'
template_data_set = 'data: [{0}].sort(function (a, b) {{return a.value - b.value;}})'
template_data = '{{value: {0}, name: \'{1}\', tooltip: {{formatter: \'{{b}}:&lt;br/&gt;{2}total: {{c}}kg\'}}}},'
template_item = '{0}kg × {1}&lt;br/&gt;'
define_parts = {
    '胸':{
        'part' : '胸',
        'left' : '25%',
        'top' : '45%',
        'center' : '[\'25%\', \'30%\']',
        'color' : '#800020',
    },
    '背':{
        'part' : '背',
        'left' : '75%',
        'top' : '45%',
        'center' : '[\'75%\', \'30%\']',
        'color' : '#CFB64A',
    },
    '手臂':{
        'part' : '手臂',
        'left' : '25%',
        'top' : '90%',
        'center' : '[\'25%\', \'75%\']',
        'color' : '#407D52',
    },
    '腿':{
        'part' : '腿',
        'left' : '75%',
        'top' : '90%',
        'center' : '[\'75%\', \'75%\']',
        'color' : '#0095B6',
    },
}
defien_part_type = ['胸', '背', '手臂', '腿']
pd.set_option('max_colwidth', 2000)
pd.set_option('display.unicode.east_asian_width', True)

In [23]:
def filter_pd_data(_type, raw):
    ret = {}
    for index, row in raw.iterrows():
        key = row[_type]
        if key not in ret:
            ret[key] = []
        ret[key].append(row)
    return ret

### Reload CSV

In [24]:
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

pd.set_option('mode.chained_assignment', None)
data = pd.read_csv('../datas/trains.csv')

# find the last sequenced index
last_index = data['ID'].max()
offset = 0
today = time.strftime('%Y/%m/%d', time.localtime(time.time() - 24 * 24 * offset))
data.iloc[:, 3][last_index:] = [today for n in range(len(data) - last_index)]

# sequence index
ids = [i + 1 for i in range(len(data))]
date = time.strftime('%Y/%m/%d', time.localtime(time.time()))
dates = [date for i in range(len(ids))]
data.iloc[:, 0] = ids
data.iloc[:, 4] = [remove_exponent(Decimal(v)) for v in data['WEIGHT']]


data.to_csv('../datas/trains.csv', index=0)

In [25]:
data = pd.read_csv('../datas/trains.csv')
data = data.dropna(axis=0, how='any')  
def generate_head(date_str):
    part1 = template_sub_text.format(define_parts[defien_part_type[0]]['part'], define_parts[defien_part_type[0]]['left'], define_parts[defien_part_type[0]]['top'])
    part2 = template_sub_text.format(define_parts[defien_part_type[1]]['part'], define_parts[defien_part_type[1]]['left'], define_parts[defien_part_type[1]]['top'])
    part3 = template_sub_text.format(define_parts[defien_part_type[2]]['part'], define_parts[defien_part_type[2]]['left'], define_parts[defien_part_type[2]]['top'])
    part4 = template_sub_text.format(define_parts[defien_part_type[3]]['part'], define_parts[defien_part_type[3]]['left'], define_parts[defien_part_type[3]]['top'])
    head = template_head.format(date_str, part1, part2, part3, part4)
    return head

def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'center')])]
    )
    return left_aligned_df


data_by_date = filter_pd_data('DATE', data)

data_formatter = {}
for key, value in data_by_date.items():
    data_set_str = ''
    center = ''
    color = ''
    df = pd.DataFrame(value)
    data_by_name = filter_pd_data('NAME', df)
    for name, row in data_by_name.items():
        total = 0
        item_str = ''
        for item in row:
            weight = item['WEIGHT']
            times = item['TIMES']
            center = define_parts[item['PART']]['center']
            color = define_parts[item['PART']]['color']
            total += weight * times
            item_str += template_item.format(weight, int(times))
        data_set_str += template_data.format(total, name, item_str)
    data_formatter[key] = template_series.format(center, template_data_set.format(data_set_str), color)
    
formatter_data = pd.DataFrame(data_formatter.items(), columns=['Date', 'Formatter'])
# formatter_data.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
left_align(formatter_data[-3:])
# display(HTML(formatter_data.to_html(index=False, justify='left')))
# formatter_data.to_html()

,Date,Formatter
39,2022/02/15,"{name: '详细信息', type: 'pie', radius: '20%', center: ['75%', '30%'], data: [{value: 3740.0, name: '宽握引体向上', tooltip: {formatter: '{b}:<br/>68.0kg × 10<br/>68.0kg × 8<br/>68.0kg × 6<br/>68.0kg × 8<br/>68.0kg × 6<br/>68.0kg × 6<br/>68.0kg × 6<br/>68.0kg × 5<br/>total: {c}kg'}},{value: 2400.0, name: '俯身杠铃划船', tooltip: {formatter: '{b}:<br/>60.0kg × 10<br/>60.0kg × 10<br/>60.0kg × 10<br/>60.0kg × 10<br/>total: {c}kg'}},{value: 3480.0, name: '坐姿拉索划船', tooltip: {formatter: '{b}:<br/>70.0kg × 12<br/>70.0kg × 12<br/>75.0kg × 12<br/>75.0kg × 12<br/>total: {c}kg'}},{value: 2800.0, name: '高位窄握下拉', tooltip: {formatter: '{b}:<br/>70.0kg × 10<br/>70.0kg × 10<br/>70.0kg × 10<br/>70.0kg × 10<br/>total: {c}kg'}},{value: 1200.0, name: '双头绳直臂下压', tooltip: {formatter: '{b}:<br/>30.0kg × 10<br/>30.0kg × 10<br/>30.0kg × 10<br/>30.0kg × 10<br/>total: {c}kg'}},{value: 1420.0, name: '俯身单臂哑铃划船', tooltip: {formatter: '{b}:<br/>40.0kg × 10<br/>40.0kg × 8<br/>35.0kg × 10<br/>35.0kg × 10<br/>total: {c}kg'}},{value: 3300.0, name: '跪姿拉索卷腹', tooltip: {formatter: '{b}:<br/>55.0kg × 20<br/>55.0kg × 20<br/>55.0kg × 20<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}), roseType: 'radius', label: {color: 'rgba(255, 255, 255, 0.3)'}, labelLine: {lineStyle: {color: 'rgba(255, 255, 255, 0.3)'}, smooth: 0.2, length: 10, length2: 20}, itemStyle: {color: '#CFB64A', shadowBlur: 200, shadowColor: 'rgba(0, 0, 0, 0.5)'}, animationType: 'scale', animationEasing: 'elasticOut', animationDelay: function (idx) {return Math.random() * 200;}},"
40,2022/02/16,"{name: '详细信息', type: 'pie', radius: '20%', center: ['25%', '75%'], data: [{value: 1200.0, name: '站姿杠铃弯举', tooltip: {formatter: '{b}:<br/>25.0kg × 10<br/>25.0kg × 10<br/>25.0kg × 10<br/>25.0kg × 10<br/>20.0kg × 10<br/>total: {c}kg'}},{value: 1740.0, name: '绳索三头下拉', tooltip: {formatter: '{b}:<br/>30.0kg × 10<br/>30.0kg × 10<br/>30.0kg × 10<br/>30.0kg × 10<br/>30.0kg × 6<br/>25.0kg × 6<br/>20.0kg × 6<br/>15.0kg × 6<br/>total: {c}kg'}},{value: 1440.0, name: '站姿拉索弯举', tooltip: {formatter: '{b}:<br/>30.0kg × 12<br/>30.0kg × 12<br/>30.0kg × 12<br/>30.0kg × 12<br/>total: {c}kg'}},{value: 1440.0, name: '固定杠臂屈伸', tooltip: {formatter: '{b}:<br/>30.0kg × 12<br/>30.0kg × 12<br/>30.0kg × 12<br/>30.0kg × 12<br/>total: {c}kg'}},{value: 1680.0, name: '坐姿哑铃推举', tooltip: {formatter: '{b}:<br/>35.0kg × 12<br/>35.0kg × 10<br/>35.0kg × 10<br/>35.0kg × 8<br/>35.0kg × 8<br/>total: {c}kg'}},{value: 1400.0, name: '站姿哑铃侧平举', tooltip: {formatter: '{b}:<br/>20.0kg × 10<br/>15.0kg × 10<br/>10.0kg × 10<br/>25.0kg × 10<br/>15.0kg × 10<br/>10.0kg × 10<br/>25.0kg × 8<br/>15.0kg × 10<br/>10.0kg × 10<br/>total: {c}kg'}},{value: 700.0, name: '站姿俯身哑铃飞鸟', tooltip: {formatter: '{b}:<br/>15.0kg × 10<br/>15.0kg × 10<br/>20.0kg × 10<br/>20.0kg × 10<br/>total: {c}kg'}},{value: 3060.0, name: '悬垂卷腹', tooltip: {formatter: '{b}:<br/>68.0kg × 15<br/>68.0kg × 15<br/>68.0kg × 15<br/>total: {c}kg'}},].sort(function (a, b) {return a.value - b.value;}), roseType: 'radius', label: {color: 'rgba(255, 255, 255, 0.3)'}, labelLine: {lineStyle: {color: 'rgba(255, 255, 255, 0.3)'}, smooth: 0.2, length: 10, length2: 20}, itemStyle: {color: '#407D52', shadowBlur: 200, shadowColor: 'rgba(0, 0, 0, 0.5)'}, animationType: 'scale', animationEasing: 'elasticOut', animationDelay: function (idx) {return Math.random() * 200;}},"
41,2022/02/21,"{name: '详细信息', type: 'pie', radius: '20%', center: ['25%', '30%'], data: [{value: 2730.0, name: '仰卧杠铃卧推', tooltip: {formatter: '{b}:<br/>60.0kg × 8<br/>60.0kg × 8<br/>60.0kg × 6<br/>60.0kg × 7<br/>60.0kg × 8<br/>70.0kg × 3<br/>50.0kg × 6<br/>total: {c}kg'}},{value: 2000.0, name: '上斜器械推举', tooltip: {formatter: '{b}:<br/>40.0kg × 10<br/>40.0kg × 10<br/>40.0kg × 10<br/>40.0kg × 10<br/>40.0kg × 10<br/>total: {c}kg'}},{value: 1440.0, name: '窄距哑铃推举', tooltip: {formatter: '{b}:<br/>30.0kg × 12<br/>30.0kg × 12<br/>30.0kg × 12<br/>30.0kg × 12<br/>total: {c}kg'}},{value: 3264.0, name: '双杠臂屈伸', tooltip: {formatter: '{

### Generate Head

In [31]:
print(generate_head('2022/02/21 - 2022/02/24'))

{% echarts 600 '95%' %}
option = {
    backgroundColor: '#2c343c', title: [{text: '2022/02/21 - 2022/02/24', left: 'center', top: 20, textStyle: {color: '#ccc'}}, {subtext: '胸', left: '25%', top: '45%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}, {subtext: '背', left: '75%', top: '45%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}, {subtext: '手臂', left: '25%', top: '90%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}, {subtext: '腿', left: '75%', top: '90%', textAlign: 'center', fontSize: 20, textStyle: {color: '#ccc'}}], tooltip: {trigger: 'item'}, visualMap: {show: false, min: 1000, max: 5000, inRange: {colorLightness: [0.8, 0.2]}},
    series: [

    ]
};
{% endecharts %}


### Generate Data by Date

In [32]:
formatter_data[formatter_data['Date'] == time.strftime('%Y/%m/%d', time.localtime(time.time()))]['Formatter']

41    {name: '详细信息', type: 'pie', radius: '20%', center: ['25%', '30%'], data: [{value: 2730.0, name: '仰卧杠铃卧推', tooltip: {formatter: '{b}:&lt;br/&gt;60.0kg × 8&lt;br/&gt;60.0kg × 8&lt;br/&gt;60.0kg × 6&lt;br/&gt;60.0kg × 7&lt;br/&gt;60.0kg × 8&lt;br/&gt;70.0kg × 3&lt;br/&gt;50.0kg × 6&lt;br/&gt;total: {c}kg'}},{value: 2000.0, name: '上斜器械推举', tooltip: {formatter: '{b}:&lt;br/&gt;40.0kg × 10&lt;br/&gt;40.0kg × 10&lt;br/&gt;40.0kg × 10&lt;br/&gt;40.0kg × 10&lt;br/&gt;40.0kg × 10&lt;br/&gt;total: {c}kg'}},{value: 1440.0, name: '窄距哑铃推举', tooltip: {formatter: '{b}:&lt;br/&gt;30.0kg × 12&lt;br/&gt;30.0kg × 12&lt;br/&gt;30.0kg × 12&lt;br/&gt;30.0kg × 12&lt;br/&gt;total: {c}kg'}},{value: 3264.0, name: '双杠臂屈伸', tooltip: {formatter: '{b}:&lt;br/&gt;68.0kg × 12&lt;br/&gt;68.0kg × 12&lt;br/&gt;68.0kg × 12&lt;br/&gt;68.0kg × 12&lt;br/&gt;total: {c}kg'}},{value: 720.0, name: '站姿高位拉锁夹胸', tooltip: {formatter: '{b}:&lt;br/&gt;30.0kg × 12&lt;br/&gt;30.0kg × 12&lt;br/&gt;total: {c}kg'}},{value: 300.0, nam

### Gallery

In [33]:
import os


example_dir = '../source/_posts/26'
gallery_template = '<div class="masonry-gallery" id="gallery">\n{0}\n</div>'
gallery_item_template = '  <div class="gallery-item">\n    <div class="content">\n      <img src="/26/{0}" />\n    </div>\n  </div>'

In [34]:
items_div = ''
for file in os.listdir(example_dir):
    items_div += gallery_item_template.format(file)
with open('../datas/gallery_scenery.xml', 'w') as w:
    w.write(gallery_template.format(items_div))

In [35]:
# -*- coding:utf-8 -*-  
data = pd.read_csv('../datas/trains.csv')
data_by_name = filter_pd_data('NAME', data)
average_name_data = {}
for name, row in data_by_name.items():
    average_name_date_data = {}
    for item in row:
        key = item['DATE']
        if key not in average_name_date_data:
            average_name_date_data[key] = { 'total' : item['WEIGHT'] * item['TIMES'], 'num' :  item['TIMES']}
        else:
            average_name_date_data[key]['total'] += item['WEIGHT'] * item['TIMES']
            average_name_date_data[key]['num'] = average_name_date_data[key]['num'] + item['TIMES']
    average_name_data[name] = average_name_date_data

filename = '../source/_posts/24/data.json'
# if not os.path.exists(filename):
#     os.mknod(filename)
with open(filename, 'w') as f:
    f.write(json.dumps(average_name_data))
# average_name_data